### Konstantina Andronikou 

## This notebook implements a Garden Path sentence 

In [16]:
from allennlp_models.pretrained import load_predictor

In [17]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [18]:
from checklist.pred_wrapper import PredictorWrapper

In [19]:
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Mary gave the child the dog bit a Band-Aid.")
output

{'verbs': [{'verb': 'gave',
   'description': '[ARG0: Mary] [V: gave] [ARG2: the child] [ARG1: the dog bit a Band - Aid] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG2',
    'I-ARG2',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']}],
 'words': ['Mary',
  'gave',
  'the',
  'child',
  'the',
  'dog',
  'bit',
  'a',
  'Band',
  '-',
  'Aid',
  '.']}

In [20]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [21]:
d = ["Mary gave the child the dog bit a Band-Aid."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'gave',
     'description': '[ARG0: Mary] [V: gave] [ARG2: the child] [ARG1: the dog bit a Band - Aid] .',
     'tags': ['B-ARG0',
      'B-V',
      'B-ARG2',
      'I-ARG2',
      'B-ARG1',
      'I-ARG1',
      'I-ARG1',
      'I-ARG1',
      'I-ARG1',
      'I-ARG1',
      'I-ARG1',
      'O']}],
   'words': ['Mary',
    'gave',
    'the',
    'child',
    'the',
    'dog',
    'bit',
    'a',
    'Band',
    '-',
    'Aid',
    '.']}],
 array([1.]))

In [22]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [24]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [25]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name1']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [26]:
editor = Editor()

# create examples
t = editor.template('Mary gave {first_name1} the dog bit a Band-Aid.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name1': 'Melissa'}, {'first_name1': 'Robert'}, {'first_name1': 'Marilyn'}, {'first_name1': 'Joan'}, {'first_name1': 'Rachel'}, {'first_name1': 'Fred'}, {'first_name1': 'Tim'}, {'first_name1': 'Betty'}, {'first_name1': 'Victoria'}, {'first_name1': 'Ryan'}, {'first_name1': 'Roy'}, {'first_name1': 'Janet'}, {'first_name1': 'Martin'}, {'first_name1': 'Alexandra'}, {'first_name1': 'David'}, {'first_name1': 'John'}, {'first_name1': 'Alice'}, {'first_name1': 'Alexandra'}, {'first_name1': 'Emily'}, {'first_name1': 'Lynn'}, {'first_name1': 'Wendy'}, {'first_name1': 'Carl'}, {'first_name1': 'Heather'}, {'first_name1': 'Louis'}, {'first_name1': 'Alan'}, {'first_name1': 'Henry'}, {'first_name1': 'Roger'}, {'first_name1': 'Alexander'}, {'first_name1': 'Kathy'}, {'first_name1': 'Julie'}, {'first_name1': 'Jason'}, {'first_name1': 'Jay'}, {'first_name1': 'Michelle'}, {'first_name1': 'Alan'}, {'first_name1': 'Michelle'}, {'first_name1': 'Simon'}, {'first_name1': 'Emma'}, {'first_name1': '

In [27]:
with open('dataset/Garden_Path.txt', 'w') as f:
    print(t.data, file = f)

In [28]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Garden_path.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    1000 (100.0%)

Example fails:
[ARG0: Mary] [V: gave] [ARG2: Michelle] [ARG1: the dog bit a Band - Aid] .
----
[ARG0: Mary] [V: gave] [ARG1: Roy] [ARG1: the dog bit a Band - Aid] .
----
[ARG0: Mary] [V: gave] [ARG2: Michelle] [ARG1: the dog bit a Band - Aid] .
----
['Mary', 'gave', 'Melissa', 'the', 'dog', 'bit', 'a', 'Band', '-', 'Aid', '.'] [False]
['Mary', 'gave', 'Robert', 'the', 'dog', 'bit', 'a', 'Band', '-', 'Aid', '.'] [False]
['Mary', 'gave', 'Marilyn', 'the', 'dog', 'bit', 'a', 'Band', '-', 'Aid', '.'] [False]
['Mary', 'gave', 'Joan', 'the', 'dog', 'bit', 'a', 'Band', '-', 'Aid', '.'] [False]
['Mary', 'gave', 'Rachel', 'the', 'dog', 'bit', 'a', 'Band', '-', 'Aid', '.'] [False]
['Mary', 'gave', 'Fred', 'the', 'dog', 'bit', 'a', 'Band', '-', 'Aid', '.'] [False]
['Mary', 'gave', 'Tim', 'the', 'dog', 'bit', 'a', 'Band', '-', 'Aid', '.'] [False]
['Mary', 'gave', 'Betty', 'the', 'dog', 'bit', 'a', 'Band', '-', 'Aid', '.

In [29]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Garden_path.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 